# Loading Data

In [1]:
train_csv_path = '../input/electricity-uk-grid-data-preparation/Wind_train.csv'
test_csv_path = '../input/electricity-uk-grid-data-preparation/Wind_test.csv'
val_csv_path = '../input/electricity-uk-grid-data-preparation/Wind_val.csv'

import pandas as pd
train_df = pd.read_csv(train_csv_path)
val_df = pd.read_csv(val_csv_path)
train_df.head()

,y(t-4),y(t-3),y(t-2),y(t-1),Hour,Day,y(t)
0,1529.0,1561.0,1573.0,1559.0,16.835556,177.0,1565.0
1,1561.0,1573.0,1559.0,1565.0,16.916944,177.0,1587.0
2,1684.0,1695.0,1750.0,1800.0,17.666944,177.0,1838.0
3,1695.0,1750.0,1800.0,1838.0,17.750556,177.0,1836.0
4,1812.0,1805.0,1769.0,1758.0,18.500278,177.0,1773.0


# Considerations

In [2]:
Similar_event_count = 100
sensitivity_consideration = 1
error_sensitivity_consideration = 1

# Error-Signal Ratio
ES_ratio = "Yes"

# Data Arranging and Normalizing

In [3]:
input_=[]
output_=[]
for row in train_df.iloc:
    input_.append((row[0:len(row)-1]).astype(float))
    output_.append(row[-1])
    
i_val=[]
o_val=[]
for row in val_df.iloc:
    i_val.append((row[0:len(row)-1]).astype(float))
    o_val.append(row[-1])
    
num_input = len(row)-1;
print(num_input)

####################################################
#
# This code is written with the help of the demo:
# https://medium.com/@benjamin.phillips22/simple-regression-with-neural-networks-in-pytorch-313f06910379
#
####################################################
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import imageio

torch.manual_seed(1)    # reproducible

x = torch.tensor(input_).float()  
y = torch.tensor(output_).float()   
new_shape = (len(y), 1)
y = y.view(new_shape)


i_val = torch.tensor(i_val).float()  
o_val = torch.tensor(o_val).float()   
new_shape = (len(o_val), 1)
o_val = o_val.view(new_shape)



max_y = torch.max(y[:,0])
min_y =torch.min(y[:,0])

max_x = torch.max(x,dim=0)
min_x = torch.min(x,dim=0)

print(max_y, min_y, max_x.values, min_x.values)

range_y = max_y - min_y
range_x = max_x.values - min_x.values

print(range_x, range_y)

    #Normalizing
x = (x - min_x.values)/range_x
y = (y - min_y)/range_y

    #Normalizing
i_val = (i_val - min_x.values)/range_x
o_val = (o_val - min_y)/range_y

6
tensor(3513.) tensor(0.) tensor([3472.0000, 3464.0000, 3473.0000, 3490.0000,   23.9208,  391.0000]) tensor([0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 2.7778e-04, 3.1000e+01])
tensor([3472.0000, 3464.0000, 3473.0000, 3490.0000,   23.9206,  360.0000]) tensor(3513.)


# Prediction NN Training

In [4]:
# torch can only train on Variable, so convert them to Variable
x, y = Variable(x), Variable(y)
 
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
        self.hidden2 = torch.nn.Linear(n_hidden, n_hidden)   # hidden layer
        self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden(x))      # activation function for hidden layer
        x = F.relu(self.hidden2(x))
        x = self.predict(x)             # linear output
        return x

# use the same net as before      
net = Net(n_feature=num_input, n_hidden=500, n_output=1)     # define the network
print(net)  # net architecture
optimizer = torch.optim.Adam(net.parameters(), lr=0.05)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss


minimum_train_loss = 1e5 #High initial values
minimum_val_loss = 1e5
EPOCH = 600

# start training
for epoch in range(EPOCH):
  
    prediction = net(x)     # input x and predict based on x

    loss = loss_func(prediction, y)     # must be (1. nn output, 2. target)

    optimizer.zero_grad()   # clear gradients for next train
    loss.backward()         # backpropagation, compute gradients
    optimizer.step()        # apply gradients
    
    if loss<minimum_train_loss:
        minimum_train_loss =loss
        net_opt = net
    
    if epoch%200 == 199:
      prediction = net_opt(i_val)
      loss_val = loss_func(prediction, o_val)
      if loss_val<minimum_val_loss:
        minimum_val_loss = loss_val
        net_opt_val = net_opt
      print ("Epoch [{}/{}], Loss: {:.4f}, Minimum Loss {:.6f}, Val Loss {:.6f}  "  .format(epoch+1, EPOCH, loss, minimum_train_loss, minimum_val_loss))

Net(
  (hidden): Linear(in_features=6, out_features=500, bias=True)
  (hidden2): Linear(in_features=500, out_features=500, bias=True)
  (predict): Linear(in_features=500, out_features=1, bias=True)
)
Epoch [200/600], Loss: 0.0002, Minimum Loss 0.000212, Val Loss 0.000192  
Epoch [400/600], Loss: 0.0001, Minimum Loss 0.000143, Val Loss 0.000130  
Epoch [600/600], Loss: 0.0001, Minimum Loss 0.000138, Val Loss 0.000130  


# Computing Test Loss

In [5]:
test_df = pd.read_csv(test_csv_path)

i_test=[]
o_test=[]
for row in test_df.iloc:
    i_test.append((row[0:len(row)-1]).astype(float))
    o_test.append(row[-1])


i_test, o_test = Variable(torch.tensor(i_test)).float(), Variable(torch.tensor(o_test).float())
new_shape = (len(o_test), 1)
o_test = o_test.view(new_shape)

    #Normalizing
i_test = (i_test - min_x.values)/range_x
o_test = (o_test - min_y)/range_y

prediction = net_opt_val(i_test)
loss_test = loss_func(prediction, o_test)

print("Normalized Test Loss",loss_test.detach().numpy())

loss_test = loss_test*range_y*range_y # As the loss function returns MSE

print("Test Loss",loss_test.detach().numpy())


Normalized Test Loss 0.00014338041
Test Loss 1769.4818


# NN for Predicting Heteroscedastic Absolute Error of Previous NN 

In [6]:
y_e = torch.zeros(len(x), dtype=torch.float) # initializing a variable of that size

for iter1 in range(len(x)):
    current_input = x[iter1,:];
    current_output = net_opt_val(current_input);
    y_e[iter1] = torch.abs(current_output - y[iter1]) 
        
x, y_e = Variable(x), Variable(y_e)
net_e = Net(n_feature=num_input, n_hidden=500, n_output=1)     # define the network
print(net_e)  # net architecture

# Error to Signal ratio
ratio_e_s = 0.5 #default
if ES_ratio == "Yes":
    ratio_e_s = torch.var(y_e)/torch.var(y)

    
print('Error to Signal ratio:', ratio_e_s)

optimizer = torch.optim.Adam(net_e.parameters(), lr=0.05)

minimum_train_loss = 1e5 #High initial values
minimum_val_loss = 1e5
EPOCH = 600


for epoch in range(EPOCH):
  
    prediction = net_e(x)     # input x and predict based on x

    loss = loss_func(prediction, y_e)     # must be (1. nn output, 2. target)

    optimizer.zero_grad()   # clear gradients for next train
    loss.backward()         # backpropagation, compute gradients
    optimizer.step()        # apply gradients
    
    if loss<minimum_train_loss:
        minimum_train_loss =loss
        net_opt = net_e
    
    if epoch%200 == 199:
        prediction = net_opt(i_val)
        loss_val = loss_func(prediction, o_val)
        if loss_val<minimum_val_loss:
            minimum_val_loss = loss_val
            net_opt_val_e = net_opt
        print ("Epoch [{}/{}], Loss: {:.4f}, Minimum Loss {:.6f}, Val Loss {:.6f}  "  .format(epoch+1, EPOCH, loss, minimum_train_loss, minimum_val_loss))
    



Net(
  (hidden): Linear(in_features=6, out_features=500, bias=True)
  (hidden2): Linear(in_features=500, out_features=500, bias=True)
  (predict): Linear(in_features=500, out_features=1, bias=True)
)
Error to Signal ratio: tensor(0.0012)


/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([20000])) that is different to the input size (torch.Size([20000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [200/600], Loss: 0.0001, Minimum Loss 0.000079, Val Loss 0.187471  
Epoch [400/600], Loss: 0.0001, Minimum Loss 0.000078, Val Loss 0.187464  
Epoch [600/600], Loss: 0.0001, Minimum Loss 0.000078, Val Loss 0.187463  


# Minimim deviation based similarity finding for Uncertainty Bounds

   Aim: To achieve all indexes of similar samples

   Steps: 

1. Itertion over all Input combinations.

2. Find all deviation by indexes 
   (sensitivity can be considered during the consideration of deviation)

3. Save top 100 similar events


In [7]:
import tensorflow as tf
import pickle as pkl
import time




max_dev_index = np.zeros(shape=(len(x),2))
Similarity = np.zeros(shape=(len(x),Similar_event_count+1))
sensitivity = torch.ones(len(max_x.values), dtype=torch.float64) # just initializing a variable of that size
error_sensitivity = torch.ones(len(max_x.values), dtype=torch.float64)
iter_debug = 0
since = time.time()

for iter1 in range(len(x)):
    current_input = x[iter1,:];
    
    # While considering only Sensitivity
    if sensitivity_consideration == 1 and error_sensitivity_consideration ==0:
        current_output = net_opt_val(current_input)
        
       # Determining Sensitivity near input[iter1]
        for iter2 in range(len(current_input)): 
            current_input[iter2] = current_input[iter2] + range_x[iter2]/1e4
            
            sensitivity[iter2] = torch.abs(current_output - net_opt_val(current_input))
            
            current_input[iter2] = current_input[iter2] - range_x[iter2]/1e4
        
        sensitivity = sensitivity/torch.max(sensitivity)
        #The highest sensitivity is considered one
    
    if sensitivity_consideration == 1 and error_sensitivity_consideration ==1:
        current_output = net_opt_val(current_input)
        current_error = net_opt_val_e(current_input)
        # ratio, ratio_e_s= 0.5 while giving equal concentration to error and signal
        
       # Determining Sensitivity and Error Sensitivity near input[iter1]
        for iter2 in range(len(current_input)): 
            current_input[iter2] = current_input[iter2] + range_x[iter2]/1e4
            
            sensitivity[iter2] = torch.abs(current_output - net_opt_val(current_input))
            error_sensitivity[iter2] = torch.abs(current_error - net_opt_val_e(current_input))
            
            current_input[iter2] = current_input[iter2] - range_x[iter2]/1e4
        
        sensitivity = sensitivity/torch.max(sensitivity)*(1-ratio_e_s)
        error_sensitivity = error_sensitivity/torch.max(error_sensitivity)* ratio_e_s
        sensitivity = sensitivity + error_sensitivity
        
        #The highest sensitivity is considered half for each, then adding
        
    
    for iter2 in range(len(x)):
        max_dev_index[iter2][0] = torch.max(sensitivity*torch.abs(current_input-x[iter2,:])/range_x).detach().numpy()
        max_dev_index[iter2][1] = iter2
        
    sort_dev_index = max_dev_index[np.argsort(max_dev_index[:, 0])]
    Similarity_threshold = sort_dev_index[Similar_event_count][0]
    matched_indexes = sort_dev_index[0:Similar_event_count,1]
    
    Similarity[iter1,0] = Similarity_threshold
    Similarity[iter1,1:(Similar_event_count+1)] = matched_indexes
    
    if iter1%1000==100:   
        print(iter1/len(x)*100, '%' '  Complete')
        time_elapsed = time.time() - since
        # Observing progres in Command Window
        # May Take about hours to find all similar events for entire training data

print('100% Complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))    

fileName = './' + 'Similarity'
fileObject = open(fileName, 'wb')

pkl.dump(Similarity, fileObject)
fileObject.close()

0.5 %  Complete
5.5 %  Complete
10.5 %  Complete
15.5 %  Complete
20.5 %  Complete
25.5 %  Complete
30.5 %  Complete
35.5 %  Complete
40.5 %  Complete
45.5 %  Complete
50.5 %  Complete
55.50000000000001 %  Complete
60.5 %  Complete
65.5 %  Complete
70.5 %  Complete
75.5 %  Complete
80.5 %  Complete
85.5 %  Complete
90.5 %  Complete
95.5 %  Complete
100% Complete in 323m 55s
